In [ ]:
%tensorflow_version 2.6

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Generic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import signal
from scipy.fft import fftshift
import scipy
from datetime import datetime

#ML Libs
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from keras import backend as K
import scipy.ndimage as nd
from keras.models import Sequential,load_model
from keras import regularizers
from keras import initializers
from keras import metrics
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, BatchNormalization, ReLU, LeakyReLU, Conv2D
from keras import layers
from keras.optimizers import adam_v2, adagrad_v2, rmsprop_v2
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, auc, roc_curve, precision_recall_fscore_support
import keras
#from keras.layers.experimental import preprocessing


Terminal = False

#Loading SQI Matched with clinitcal to the Dataframe

In [ ]:
#Loading SQI Matched with clinical to the Dataframe

id = '1DDf_vluJE-zg--41A0zpP5_9-FU-wsJp' # The shareable link id
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Complete_SQIs_with_Clinical.csv')  
SQI_C = pd.read_csv('Complete_SQIs_with_Clinical.csv')


id = '1zEpiX0yXT16cz3znpo8wjoV_U_8vSoyb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Raw_signals.csv')  
Raw = pd.read_csv('Raw_signals.csv')

if Terminal:
    print("\n SQI with Clinical Match:")
    print(SQI_C)
    print("\n Raw Signals")
    print(Raw)


#Not really need, here for completeness
event = ['event_shock', 'reshock24','diagnosis_admission',\
     'ascites', 'respiratory_distress', 'ventilation_cannula', \
     'ventilation_mechanical', 'ventilation_ncpap', 'bleeding_severe', \
     'cns_abnormal', 'liver_mild', 'pleural_effusion', 'skidney']

event_shock = 'shock_admission'

SQI_C['keep'] = False
for i in range(len(event)):
    event_s = event[i]
    SQI_C['keep'][SQI_C[event_s] == True] = True
    #print("\n Total ", event[i], " events:")

In [ ]:
#Mixed, Admitted vs Not, Pre vs Post
#Patient 1 pre shock => 2 Hours Each
#Patient 2 post shock, in this case P1 and P2 are the same
P1times = datetime.strptime('1809202015450000000', '%d%m%Y%H%M%S%f')##########
P1timef = datetime.strptime('1809202017450000000', '%d%m%Y%H%M%S%f') 

P2times = datetime.strptime('18092020181500000', '%d%m%Y%H%M%S%f')##########
P2timef = datetime.strptime('18092020201500000', '%d%m%Y%H%M%S%f')

#Patient 3 had no shock
#Patient 4 had shock 
P3times = datetime.strptime('20072020143000000', '%d%m%Y%H%M%S%f')
P3timef = datetime.strptime('20072020163000000', '%d%m%Y%H%M%S%f')

P4times = datetime.strptime('03112020103000000', '%d%m%Y%H%M%S%f')
P4timef = datetime.strptime('03112020123000020', '%d%m%Y%H%M%S%f') #added 20 miliseconds to match the length of Patient 1


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

Patient1 = Raw[(Raw.study_no == '003-2104') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2104') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]
Patient3 = Raw[(Raw.study_no == '003-2162') & (P3times <= Raw.PPG_Datetime) & (P3timef > Raw.PPG_Datetime)]
Patient4 = Raw[(Raw.study_no == '003-2028') & (P4times <= Raw.PPG_Datetime) & (P4timef > Raw.PPG_Datetime)]
#Patient3 = Raw[(Raw.study_no == '003-2104') & (P3times <= Raw.PPG_Datetime) & (P3timef > Raw.PPG_Datetime)]#############


# Patient1['Label'] = 0 
# Patient2['Label'] = 1
# Patient3['Label'] = 0 
# Patient4['Label'] = 1
#Patient3['Label'] = 1

if Terminal:
  print(Patient1)
  print(Patient2)
  print(Patient3)
  print(Patient4)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 100)
Patient2_w = np.array_split(Patient2, 100)
Patient3_w = np.array_split(Patient3, 100)
Patient4_w = np.array_split(Patient4, 100)
#Patient3_w = np.array_split(Patient3, 50)##########

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient_w),7200,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'PLETH_bpf')#Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'PLETH_bpf')
Patient3_w_arr = choose_signal(Patient3_w, 'PLETH_bpf') 
Patient4_w_arr = choose_signal(Patient4_w, 'PLETH_bpf')
#Patient3_w_arr = choose_signal(Patient3_w, 'IR_ADC_bpf')#############


Patient1_w_arr_l = np.zeros((100,1), dtype=int)
Patient2_w_arr_l = np.ones((100,1), dtype=int)
Patient3_w_arr_l = np.full((100,1),2, dtype=int)
Patient4_w_arr_l = np.full((100,1),3, dtype=int)
#Patient3_w_arr_l = np.ones((100,1), dtype=int)############

#Defining Sampling rate
fs = 100

Whole_Array_Test = np.empty((30,7))

Score_array = np.empty((5,1))
Recall = np.empty((5,1))
Precision = np.empty((5,1))
F1S = np.empty((5,1))

##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr,Patient2_w_arr,Patient3_w_arr,Patient4_w_arr))
Labels = np.vstack((Patient1_w_arr_l,Patient2_w_arr_l,Patient3_w_arr_l,Patient4_w_arr_l))

for i in range(10):
#counter2 = 0
#for jj in range(20,300,20):
  counter = 0 
  k = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
  for train_index, test_index in k.split(Data,Labels):
    print("Fold Number =================================================== ", counter)
    X_train = Data[train_index]
    X_test = Data[test_index]
    y_train = Labels[train_index]
    y_test = Labels[test_index]

    if Terminal:
      print(train_index)
      print(test_index)
    #Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

    #Training_sets = np.vstack((P1_tr,P2_tr))
    Globalmean = np.mean(X_train)
    Globalstd = np.std(X_train)


    P_train = (X_train - Globalmean) / Globalstd
    P_test = (X_test - Globalmean) / Globalstd

    #Empty arrays
    #P1Sxx = np.empty((len(P1_training),26,32))
    #P1Sxxtest = np.empty((len(P1_test),26,32))
    PSxx = np.empty((len(P_train),23,32))
    PSxxtest = np.empty((len(P_test),23,32))


    #Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
    #Patient 1

    def spectrogram_arr(Input, Output_arr, fs):
      for i in range(len(Input)):
        f, t, Sxx = signal.spectrogram(Input[i], fs)
        fmin = 1 # Hz
        fmax = 10 # Hz
        freq_slice = np.where((f >= fmin) & (f <= fmax))
        # keep only frequencies of interest
        f   = f[freq_slice]
        Sxx = Sxx[freq_slice,:][0]
        Output_arr[i] = Sxx
      return Output_arr

    X_train = spectrogram_arr(P_train, PSxx, fs)
    X_test = spectrogram_arr(P_test, PSxxtest, fs)

    Terminal = False

    if Terminal:
      print(X_train.shape)
      print(y_train.shape)
      print(X_test.shape)
      print(y_test.shape)



    #MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
    #tf.random.set_seed(1234)

    #converting to categorical
    y_trainf = tf.keras.utils.to_categorical(y_train)
    y_testf = tf.keras.utils.to_categorical(y_test)
    #X_trainf = X_train.reshape(-1, 26*32)
    #X_testf = X_test.reshape(-1, 26*32)

    from sklearn.decomposition import PCA


    #print(X_train)
    print(X_test.shape)
    pca = PCA(n_components=60) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
    X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
    pca = pca.fit(X_train_2d)
    X_trainf = pca.transform(X_train_2d)
    #pca = PCA(n_components=15)
    #pca = pca.fit(X_test_2d)
    X_testf = pca.transform(X_test_2d)
    # for i in range(X_train.shape[0]):
    #   pca = pca.fit(X_train[i,:,:])
    #   X_trainf = pca.transform(X_train[i,:,:])
    # for i in range(X_test.shape[0]):
    #   pca = pca.fit(X_test[i,:,:])
    #   X_testf = pca.transform(X_test[i,:,:])

    Terminal = True
    
    print(X_trainf.shape)
    print(X_testf.shape)
    
    if Terminal:
      print('Train Labels')
      print(y_trainf.shape)
      print('Train Data')
      print(X_trainf.shape)

    # Simple ANN Start ========================================================================================

    # model = Sequential()
    # model.add(Dense(16,'relu', input_shape=(15,)))#,use_bias=True, kernel_initializer='glorot_uniform'))
    # #model.add(BatchNormalization())
    # #model.add(Dense(32,'relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(64,'relu'))
    # model.add(Dropout(0.4))
    # model.add(Dense(16,'relu'))
    # model.add(Dropout(0.4))
    # model.add(Dense(4))
    # model.add(Activation('Softmax'))

    from keras.regularizers import l1_l2
    # model = Sequential()
    # model.add(Dense(64,'relu', input_shape=(60,), kernel_initializer='glorot_uniform'))#,use_bias=True, kernel_initializer='glorot_uniform'))
    # #model.add(BatchNormalization())
    # model.add(Dense(128,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    # model.add(Dropout(0.4))
    # model.add(Dense(256,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    # model.add(Dropout(0.3))
    # model.add(Dense(128,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))
    # model.add(Dropout(0.3))
    # model.add(Dense(64,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))
    # model.add(Dropout(0.4))
    # model.add(Dense(64,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))
    # #model.add(Dropout(0.2))
    # model.add(Dense(4))
    # model.add(Activation('Softmax'))

    # model = Sequential()
    # model.add(Dense(32,'relu', input_shape=(60,), kernel_initializer='glorot_uniform'))#,use_bias=True, kernel_initializer='glorot_uniform'))
    # model.add(Dropout(0.6))
    # model.add(Dense(512,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    # model.add(Dense(512,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    # model.add(Dropout(0.6))
    # model.add(Dense(256,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    # model.add(Dense(4))
    # model.add(Activation('Softmax'))

    # # . . . 
    # #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
    # model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    # history = model.fit(X_trainf, y_trainf, epochs=150, batch_size=32, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])

    model = Sequential()
    model.add(Dense(64,'relu', input_shape=(60,), kernel_initializer='glorot_uniform'))#,use_bias=True, kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.6))
    model.add(Dense(128,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    model.add(Dense(128,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.6))
    model.add(Dense(64,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    model.add(Dense(4))
    model.add(Activation('Softmax'))

    # . . . 
    #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
    model.compile(optimizer=adam_v2.Adam(learning_rate=3e-3),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model.fit(X_trainf, y_trainf, epochs=150, batch_size=32, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])



    # ANN ==========================================================================================

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Accuracy and Loss')
    plt.ylabel('accuracy/loss')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
    plt.show()

    score = model.evaluate(X_testf, y_testf)

    y_prob= model.predict(X_testf)
    y_pred_unseen = y_prob.argmax(axis=-1)

    PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

    print('\nConfusion Matrix on Unseen Data:')
    print(confusion_matrix(y_test, y_pred_unseen))

    print('\nReport on Unseen Data:')
    print(classification_report(y_test, y_pred_unseen))


    #Sourced from https://www.codegrepper.com/code-examples/python/frameworks/-file-path-python/roc+python+example
    # probs = model.predict_proba(X_testf)
    # preds = probs[:,1]
    # fpr, tpr, threshold = roc_curve(y_test, preds)
    # roc_auc = auc(fpr, tpr)

    # plt.title('Receiver Operating Characteristic on Unseen Data')
    # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    # plt.legend(loc = 'lower right')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()


    Score_array[counter] = score[1]
    Recall[counter] = PRF[1]
    Precision[counter] = PRF[0] 
    F1S[counter] =  PRF[2]
    counter = counter + 1



  print(Score_array)

  print("\n K-Fold CV Max Test Accuracy:")
  print(np.max(Score_array))

  print("\n K-Fold CV Min Test Accuracy:")
  print(np.min(Score_array))

  print("\n K-Fold CV Average Test Accuracy:")
  print(np.sum(Score_array)/5)
  print("\n K-Fold CV Average Test Recall:")
  print(np.sum(Recall)/5)
  print("\n K-Fold CV Average Test Precision:")
  print(np.sum(Precision)/5)
  print("\n K-Fold CV Average Test F1Score:")
  print(np.sum(F1S)/5)

  # Whole_Array_Test[counter2][0]= jj
  # Whole_Array_Test[counter2][1]= np.max(Score_array)
  # Whole_Array_Test[counter2][2]= np.min(Score_array)
  # Whole_Array_Test[counter2][3]= np.sum(Score_array)/5
  # Whole_Array_Test[counter2][4]= np.sum(Recall)/5
  # Whole_Array_Test[counter2][5]= np.sum(Precision)/5
  # Whole_Array_Test[counter2][6]= np.sum(F1S)/5
    
    
  #   print(Whole_Array_Test[counter2])
      
  #   counter2 = counter2 + 1

  # print(Whole_Array_Test)
  # Results = pd.DataFrame(Whole_Array_Test)
  # Results.to_csv('Model1PCA.csv') 
  # files.download('Model1PCA.csv')

In [ ]:
from google.colab import files
Results = pd.DataFrame(Whole_Array_Test)
Results.to_csv('Model2PCA.csv') 
files.download('Model2PCA.csv')

In [ ]:
#SHOCK ADMISSION VS SHOCK NOT ADMISSION  # 2 Hour Samples Each
#Patient 1 had no shock
#Patient 2 had shock 
P1times = datetime.strptime('20072020143000000', '%d%m%Y%H%M%S%f')
P1timef = datetime.strptime('20072020163000000', '%d%m%Y%H%M%S%f')

P2times = datetime.strptime('03112020103000000', '%d%m%Y%H%M%S%f')
P2timef = datetime.strptime('03112020123000020', '%d%m%Y%H%M%S%f') #added 20 miliseconds to match the length of Patient 1

#P3times = datetime.strptime('18092020203000000', '%d%m%Y%H%M%S%f')##########
#P3timef = datetime.strptime('18092020213000000', '%d%m%Y%H%M%S%f') 


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

Patient1 = Raw[(Raw.study_no == '003-2162') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2028') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]
#Patient3 = Raw[(Raw.study_no == '003-2104') & (P3times <= Raw.PPG_Datetime) & (P3timef > Raw.PPG_Datetime)]#############


Patient1['Label'] = 0 
Patient2['Label'] = 1
#Patient3['Label'] = 1

if Terminal:
  print(Patient1)
  print(Patient2)
  #print(Patient3)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 100)
Patient2_w = np.array_split(Patient2, 100)
#Patient3_w = np.array_split(Patient3, 50)##########

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient_w),7200,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'PLETH_bpf') #Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'PLETH_bpf')
#Patient3_w_arr = choose_signal(Patient3_w, 'IR_ADC_bpf')#############


Patient1_w_arr_l = np.zeros((100,1), dtype=int)
Patient2_w_arr_l = np.ones((100,1), dtype=int)
#Patient3_w_arr_l = np.ones((100,1), dtype=int)############

#Defining Sampling rate
fs = 100

Score_array = np.empty((5,1))


# Whole_Array_Test = np.empty((30,7))

Score_array = np.empty((5,1))
Recall = np.empty((5,1))
Precision = np.empty((5,1))
F1S = np.empty((5,1))

##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr,Patient2_w_arr))
Labels = np.vstack((Patient1_w_arr_l,Patient2_w_arr_l))


# counter2 = 0
# for jj in range(20,180,20):
for i in range(10):
  counter = 0
  k = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
  for train_index, test_index in k.split(Data,Labels):
    print("Fold Number =================================================== ", counter)
    X_train = Data[train_index]
    X_test = Data[test_index]
    y_train = Labels[train_index]
    y_test = Labels[test_index]

    if Terminal:
      print(train_index)
      print(test_index)
    #Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

    #Training_sets = np.vstack((P1_tr,P2_tr))
    Globalmean = np.mean(X_train)
    Globalstd = np.std(X_train)


    P_train = (X_train - Globalmean) / Globalstd
    P_test = (X_test - Globalmean) / Globalstd

    #Empty arrays
    #P1Sxx = np.empty((len(P1_training),26,32))
    #P1Sxxtest = np.empty((len(P1_test),26,32))
    PSxx = np.empty((len(P_train),26,32))
    PSxxtest = np.empty((len(P_test),26,32))


    #Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
    #Patient 1

    def spectrogram_arr(Input, Output_arr, fs):
      for i in range(len(Input)):
        f, t, Sxx = signal.spectrogram(Input[i], fs)
        fmin = 0 # Hz
        fmax = 10 # Hz
        freq_slice = np.where((f >= fmin) & (f <= fmax))
        # keep only frequencies of interest
        f   = f[freq_slice]
        Sxx = Sxx[freq_slice,:][0]
        Output_arr[i] = Sxx
      return Output_arr

    X_train = spectrogram_arr(P_train, PSxx, fs)
    X_test = spectrogram_arr(P_test, PSxxtest, fs)

    Terminal = False

    if Terminal:
      print(X_train.shape)
      print(y_train.shape)
      print(X_test.shape)
      print(y_test.shape)



    #MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
    #tf.random.set_seed(1234)

    #converting to categorical
    y_trainf = tf.keras.utils.to_categorical(y_train)
    y_testf = tf.keras.utils.to_categorical(y_test)
    #X_trainf = X_train.reshape(-1, 26*32)
    #X_testf = X_test.reshape(-1, 26*32)

    from sklearn.decomposition import PCA


    # #print(X_train)
    # print(X_test.shape)
    # pca = PCA(n_components=17) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    # for i in range(X_train.shape[2]):
    #   pca = pca.fit(X_train[:,:,i])
    #   X_trainf = pca.transform(X_train[:,:,i])
    #   X_testf = pca.transform(X_test[:,:,i])

    pca = PCA(n_components=120) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
    X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
    pca = pca.fit(X_train_2d)
    X_trainf = pca.transform(X_train_2d)
    X_testf = pca.transform(X_test_2d)

    Terminal = True
    print(X_trainf.shape)
    print(X_testf.shape)
    
    if Terminal:
      print('Train Labels')
      print(y_trainf.shape)
      print('Train Data')
      print(X_train.shape)

    # Simple ANN Start ========================================================================================



    model = Sequential()
    model.add(Dense(64,'relu', input_shape=(120,), kernel_initializer='glorot_uniform'))#,use_bias=True, kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.4))
    model.add(Dense(128,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.4))
    model.add(Dense(128,'relu', bias_regularizer=l2(0.001), kernel_initializer='glorot_uniform'))#, kernel_regularizer=l2(0.01)))
    model.add(Dense(2))
    model.add(Activation('Softmax'))

    # . . . 
    #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
    model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model.fit(X_trainf, y_trainf, epochs=200, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])
    # #500 epochs for IR_ADC

    # ANN ==========================================================================================

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Accuracy and Loss')
    plt.ylabel('accuracy/loss')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
    plt.show()

    score = model.evaluate(X_testf, y_testf)

    y_prob= model.predict(X_testf)
    y_pred_unseen = y_prob.argmax(axis=-1)

    PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

    print('\nConfusion Matrix on Unseen Data:')
    print(confusion_matrix(y_test, y_pred_unseen))

    print('\nReport on Unseen Data:')
    print(classification_report(y_test, y_pred_unseen))


    #Sourced from https://www.codegrepper.com/code-examples/python/frameworks/-file-path-python/roc+python+example
    # probs = model.predict_proba(X_testf)
    # preds = probs[:,1]
    # fpr, tpr, threshold = roc_curve(y_test, preds)
    # roc_auc = auc(fpr, tpr)

    # plt.title('Receiver Operating Characteristic on Unseen Data')
    # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    # plt.legend(loc = 'lower right')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()

    Score_array[counter] = score[1]
    Recall[counter] = PRF[1]
    Precision[counter] = PRF[0] 
    F1S[counter] =  PRF[2]
    counter = counter + 1

  print(Score_array)

  print("\n K-Fold CV Max Test Accuracy:")
  print(np.max(Score_array))

  print("\n K-Fold CV Min Test Accuracy:")
  print(np.min(Score_array))

  print("\n K-Fold CV Average Test Accuracy:")
  print(np.sum(Score_array)/5)
  print("\n K-Fold CV Average Test Recall:")
  print(np.sum(Recall)/5)
  print("\n K-Fold CV Average Test Precision:")
  print(np.sum(Precision)/5)
  print("\n K-Fold CV Average Test F1Score:")
  print(np.sum(F1S)/5)

  #   Whole_Array_Test[counter2][0]= jj
  #   Whole_Array_Test[counter2][1]= np.max(Score_array)
  #   Whole_Array_Test[counter2][2]= np.min(Score_array)
  #   Whole_Array_Test[counter2][3]= np.sum(Score_array)/5
  #   Whole_Array_Test[counter2][4]= np.sum(Recall)/5
  #   Whole_Array_Test[counter2][5]= np.sum(Precision)/5
  #   Whole_Array_Test[counter2][6]= np.sum(F1S)/5

  #   print(Whole_Array_Test[counter2])
      
  #   counter2 = counter2 + 1

  # print(Whole_Array_Test)
  # Results = pd.DataFrame(Whole_Array_Test)
  # Results.to_csv('Model1PCA.csv') 
  # files.download('Model1PCA.csv')

In [ ]:
#SHOCK  VS NO SHOCK without Shock In Admission # 2 Hour Samples Each
#Patient 1 pre shock
#Patient 2 post shock, in this case P1 and P2 are the same
P1times = datetime.strptime('1809202015450000000', '%d%m%Y%H%M%S%f')##########
P1timef = datetime.strptime('1809202017450000000', '%d%m%Y%H%M%S%f') 

P2times = datetime.strptime('18092020181500000', '%d%m%Y%H%M%S%f')##########
P2timef = datetime.strptime('18092020201500000', '%d%m%Y%H%M%S%f') 


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

Patient1 = Raw[(Raw.study_no == '003-2104') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2104') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]


Patient1['Label'] = 0 
Patient2['Label'] = 1
#Patient3['Label'] = 1

if Terminal:
  print(Patient1)
  print(Patient2)
  #print(Patient3)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 100)
Patient2_w = np.array_split(Patient2, 100)
#Patient3_w = np.array_split(Patient3, 50)##########

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient_w),7200,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'PLETH_bpf') #Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'PLETH_bpf')
#Patient3_w_arr = choose_signal(Patient3_w, 'IR_ADC_bpf')#############


Patient1_w_arr_l = np.zeros((100,1), dtype=int)
Patient2_w_arr_l = np.ones((100,1), dtype=int)
#Patient3_w_arr_l = np.ones((100,1), dtype=int)############

#Defining Sampling rate
fs = 100
Whole_Array_Test = np.empty((30,7))
Score_array = np.empty((5,1))
Recall = np.empty((5,1))
Precision = np.empty((5,1))
F1S = np.empty((5,1))

##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr,Patient2_w_arr))
Labels = np.vstack((Patient1_w_arr_l,Patient2_w_arr_l))


counter2 = 0
for jj in range(20,180,20):
  counter = 0
  k = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
  for train_index, test_index in k.split(Data,Labels):
    print("Fold Number =================================================== ", counter)
    X_train = Data[train_index]
    X_test = Data[test_index]
    y_train = Labels[train_index]
    y_test = Labels[test_index]

    if Terminal:
      print(train_index)
      print(test_index)
    #Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

    #Training_sets = np.vstack((P1_tr,P2_tr))
    Globalmean = np.mean(X_train)
    Globalstd = np.std(X_train)


    P_train = (X_train - Globalmean) / Globalstd
    P_test = (X_test - Globalmean) / Globalstd

    #Empty arrays
    #P1Sxx = np.empty((len(P1_training),26,32))
    #P1Sxxtest = np.empty((len(P1_test),26,32))
    PSxx = np.empty((len(P_train),26,32))
    PSxxtest = np.empty((len(P_test),26,32))


    #Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
    #Patient 1

    def spectrogram_arr(Input, Output_arr, fs):
      for i in range(len(Input)):
        f, t, Sxx = signal.spectrogram(Input[i], fs)
        fmin = 0 # Hz
        fmax = 10 # Hz
        freq_slice = np.where((f >= fmin) & (f <= fmax))
        # keep only frequencies of interest
        f   = f[freq_slice]
        Sxx = Sxx[freq_slice,:][0]
        Output_arr[i] = Sxx
      return Output_arr

    X_train = spectrogram_arr(P_train, PSxx, fs)
    X_test = spectrogram_arr(P_test, PSxxtest, fs)

    Terminal = False

    if Terminal:
      print(X_train.shape)
      print(y_train.shape)
      print(X_test.shape)
      print(y_test.shape)



    #MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
    #tf.random.set_seed(1234)

    #converting to categorical
    y_trainf = tf.keras.utils.to_categorical(y_train)
    y_testf = tf.keras.utils.to_categorical(y_test)
    #X_trainf = X_train.reshape(-1, 26*32)
    #X_testf = X_test.reshape(-1, 26*32)

    from sklearn.decomposition import PCA


    # #print(X_train)
    # print(X_test.shape)
    # pca = PCA(n_components=23) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    # for i in range(X_train.shape[2]):
    #   pca = pca.fit(X_train[:,:,i])
    #   X_trainf = pca.transform(X_train[:,:,i])
    #   X_testf = pca.transform(X_test[:,:,i])

    pca = PCA(n_components=jj) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
    X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
    pca = pca.fit(X_train_2d)
    X_trainf = pca.transform(X_train_2d)
    X_testf = pca.transform(X_test_2d)

    Terminal = True
    print(X_trainf.shape)
    print(X_testf.shape)
    
    if Terminal:
      print('Train Labels')
      print(y_trainf.shape)
      print('Train Data')
      print(X_train.shape)

    # Simple ANN Start ========================================================================================

    model = Sequential()
    model.add(Dense(128,'relu', input_shape=(jj,)))#,use_bias=True, kernel_initializer='glorot_uniform'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(256,'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64,'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2))
    model.add(Activation('Softmax'))

    # . . . 
    #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
    model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model.fit(X_trainf, y_trainf, epochs=150, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])


    # ANN ==========================================================================================

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Accuracy and Loss')
    plt.ylabel('accuracy/loss')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
    plt.show()

    score = model.evaluate(X_testf, y_testf)

    y_prob= model.predict(X_testf)
    y_pred_unseen = y_prob.argmax(axis=-1)

    PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

    print('\nConfusion Matrix on Unseen Data:')
    print(confusion_matrix(y_test, y_pred_unseen))

    print('\nReport on Unseen Data:')
    print(classification_report(y_test, y_pred_unseen))


    #Sourced from https://www.codegrepper.com/code-examples/python/frameworks/-file-path-python/roc+python+example
    # probs = model.predict_proba(X_testf)
    # preds = probs[:,1]
    # fpr, tpr, threshold = roc_curve(y_test, preds)
    # roc_auc = auc(fpr, tpr)

    # plt.title('Receiver Operating Characteristic on Unseen Data')
    # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    # plt.legend(loc = 'lower right')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()

    Score_array[counter] = score[1]
    Recall[counter] = PRF[1]
    Precision[counter] = PRF[0] 
    F1S[counter] =  PRF[2]
    counter = counter + 1

  print(Score_array)

  print("\n K-Fold CV Max Test Accuracy:")
  print(np.max(Score_array))

  print("\n K-Fold CV Min Test Accuracy:")
  print(np.min(Score_array))

  print("\n K-Fold CV Average Test Accuracy:")
  print(np.sum(Score_array)/5)
  print("\n K-Fold CV Average Test Recall:")
  print(np.sum(Recall)/5)
  print("\n K-Fold CV Average Test Precision:")
  print(np.sum(Precision)/5)
  print("\n K-Fold CV Average Test F1Score:")
  print(np.sum(F1S)/5)

  Whole_Array_Test[counter2][0]= jj
  Whole_Array_Test[counter2][1]= np.max(Score_array)
  Whole_Array_Test[counter2][2]= np.min(Score_array)
  Whole_Array_Test[counter2][3]= np.sum(Score_array)/5
  Whole_Array_Test[counter2][4]= np.sum(Recall)/5
  Whole_Array_Test[counter2][5]= np.sum(Precision)/5
  Whole_Array_Test[counter2][6]= np.sum(F1S)/5

  print(Whole_Array_Test[counter2])
    
  counter2 = counter2 + 1

print(Whole_Array_Test)
Results = pd.DataFrame(Whole_Array_Test)
Results.to_csv('Model3PCA.csv') 
files.download('Model3PCA.csv')

In [ ]:
#Patient 1 pre shock (Admitted with Shock)
#Patient 2 post shock, in this case P1 and P2 are the same 2009
#Unfortunately Post shock data is much shorter and therefore we split it into smaller windows (Data is still less)
Terminal = False
if Terminal:
  #Print the Study
  print('Study 003-2009 Datetime:')
  print(Raw[(Raw.study_no == '003-2009')].PPG_Datetime)

  #Printing Time of Event
  print('PPG Shock Event Start -2009:')
  print(SQI_C[(SQI_C.study_no == '003-2009') & (SQI_C.keep == True)].PPG_w_s)
  print('PPG Shock Event End -2009:')
  print(SQI_C[(SQI_C.study_no == '003-2009') & (SQI_C.keep == True)].PPG_w_f)

  #Print the Study
  print('Study 003-2103 Datetime:')
  print(Raw[(Raw.study_no == '003-2103')].PPG_Datetime)

  #Printing Time of Event
  print('PPG Shock Event Start -2103:')
  print(SQI_C[(SQI_C.study_no == '003-2103') & (SQI_C.keep == True)].PPG_w_s)
  print('PPG Shock Event End -2103:')
  print(SQI_C[(SQI_C.study_no == '003-2103') & (SQI_C.keep == True)].PPG_w_f)


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

P1times = datetime.strptime('28072020160400000', '%d%m%Y%H%M%S%f')########## Start Until Shock
P1timef = datetime.strptime('29072020032820200', '%d%m%Y%H%M%S%f') 

P2times = datetime.strptime('29072020033146280', '%d%m%Y%H%M%S%f')########## Shock Until the End
P2timef = datetime.strptime('29072020044900000', '%d%m%Y%H%M%S%f') 

# P3times = datetime.strptime('29072020033146280', '%d%m%Y%H%M%S%f')########## Shock Until the End for Patient 2103
# P3timef = datetime.strptime('18092020044800000', '%d%m%Y%H%M%S%f') 

#Extracting
Patient1 = Raw[(Raw.study_no == '003-2009') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2009') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]
#Patient3 = Raw[(Raw.study_no == '003-2103') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]


#Unnecessary Label Added
Patient1['Label'] = 0 
Patient2['Label'] = 1
#Patient3['Label'] = 1

if Terminal:
  print(Patient1)
  print(Patient2)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 684) #Split into 60 Sec Windows
Patient2_w = np.array_split(Patient2, 77)
#Patient3_w = np.array_split(Patient3, 50)##########

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient_w),6000,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'IR_ADC_bpf') #Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'IR_ADC_bpf')


Patient1_w_arr_l = np.zeros((684,1), dtype=int)
Patient2_w_arr_l = np.ones((77,1), dtype=int)

#Defining Sampling rate
fs = 100

Score_array = np.empty((5,1))
Whole_Array_Test = np.empty((30,7))

##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr[-77:,:],Patient2_w_arr)) #for a balanced model we use slicing 
Labels = np.vstack((Patient1_w_arr_l[-77:,:],Patient2_w_arr_l))

Recall = np.empty((5,1))
Precision = np.empty((5,1))
F1S = np.empty((5,1))


counter2 = 0
for jj in range(80,81,80):
  counter = 0
  k = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
  for train_index, test_index in k.split(Data,Labels):
    print("Fold Number =================================================== ", counter)
    X_train = Data[train_index]
    X_test = Data[test_index]
    y_train = Labels[train_index]
    y_test = Labels[test_index]

    if Terminal:
      print(train_index)
      print(test_index)
    #Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

    #Training_sets = np.vstack((P1_tr,P2_tr))
    Globalmean = np.mean(X_train)
    Globalstd = np.std(X_train)


    P_train = (X_train - Globalmean) / Globalstd
    P_test = (X_test - Globalmean) / Globalstd

    #Empty arrays
    #P1Sxx = np.empty((len(P1_training),26,32))
    #P1Sxxtest = np.empty((len(P1_test),26,32))
    PSxx = np.empty((len(P_train),26,26))
    PSxxtest = np.empty((len(P_test),26,26))


    #Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
    #Patient 1

    def spectrogram_arr(Input, Output_arr, fs):
      for i in range(len(Input)):
        f, t, Sxx = signal.spectrogram(Input[i], fs)
        fmin = 0 # Hz
        fmax = 10 # Hz
        freq_slice = np.where((f >= fmin) & (f <= fmax))
        # keep only frequencies of interest
        f   = f[freq_slice]
        Sxx = Sxx[freq_slice,:][0]
        Output_arr[i] = Sxx
      return Output_arr

    X_train = spectrogram_arr(P_train, PSxx, fs)
    X_test = spectrogram_arr(P_test, PSxxtest, fs)

    Terminal = False

    if Terminal:
      print(X_train.shape)
      print(y_train.shape)
      print(X_test.shape)
      print(y_test.shape)



    #MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
    #tf.random.set_seed(1234)

    #converting to categorical
    y_trainf = tf.keras.utils.to_categorical(y_train)
    y_testf = tf.keras.utils.to_categorical(y_test)
    #X_trainf = X_train.reshape(-1, 26*32)
    #X_testf = X_test.reshape(-1, 26*32)

    from sklearn.decomposition import PCA


    # #print(X_train)
    # print(X_test.shape)
    # pca = PCA(n_components=26) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    # for i in range(X_train.shape[2]):
    #   pca = pca.fit(X_train[:,:,i])
    #   X_trainf = pca.transform(X_train[:,:,i])
    #   X_testf = pca.transform(X_test[:,:,i])

    pca = PCA(n_components=jj) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
    X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
    pca = pca.fit(X_train_2d)
    X_trainf = pca.transform(X_train_2d)
    X_testf = pca.transform(X_test_2d)

    Terminal = True
    print(X_trainf.shape)
    print(X_testf.shape)
    
    if Terminal:
      print('Train Labels')
      print(y_trainf.shape)
      print('Train Data')
      print(X_train.shape)

    # Simple ANN Start ========================================================================================

    model = Sequential()
    model.add(Dense(256,'relu', input_shape=(jj,), kernel_initializer ='glorot_uniform' ))
    #model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(512,'relu'))
    model.add(Dropout(0.4))
    model.add(Dense(256,'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('Softmax'))

    # . . . 
    #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
    model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.summary()
    history = model.fit(X_trainf, y_trainf, epochs=600, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])


    # ANN ==========================================================================================

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Accuracy and Loss')
    plt.ylabel('accuracy/loss')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
    plt.show()

    score = model.evaluate(X_testf, y_testf)

    y_prob= model.predict(X_testf)
    y_pred_unseen = y_prob.argmax(axis=-1)

    PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

    print('\nConfusion Matrix on Unseen Data:')
    print(confusion_matrix(y_test, y_pred_unseen))

    print('\nReport on Unseen Data:')
    print(classification_report(y_test, y_pred_unseen))


    #Sourced from https://www.codegrepper.com/code-examples/python/frameworks/-file-path-python/roc+python+example
    # probs = model.predict_proba(X_testf)
    # preds = probs[:,1]
    # fpr, tpr, threshold = roc_curve(y_test, preds)
    # roc_auc = auc(fpr, tpr)

    # plt.title('Receiver Operating Characteristic on Unseen Data')
    # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    # plt.legend(loc = 'lower right')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()

    Score_array[counter] = score[1]
    Recall[counter] = PRF[1]
    Precision[counter] = PRF[0] 
    F1S[counter] =  PRF[2]
    counter = counter + 1

  print(Score_array)

  print("\n K-Fold CV Max Test Accuracy:")
  print(np.max(Score_array))

  print("\n K-Fold CV Min Test Accuracy:")
  print(np.min(Score_array))

  print("\n K-Fold CV Average Test Accuracy:")
  print(np.sum(Score_array)/5)
  print("\n K-Fold CV Average Test Recall:")
  print(np.sum(Recall)/5)
  print("\n K-Fold CV Average Test Precision:")
  print(np.sum(Precision)/5)
  print("\n K-Fold CV Average Test F1Score:")
  print(np.sum(F1S)/5)

  Whole_Array_Test[counter2][0]= jj
  Whole_Array_Test[counter2][1]= np.max(Score_array)
  Whole_Array_Test[counter2][2]= np.min(Score_array)
  Whole_Array_Test[counter2][3]= np.sum(Score_array)/5
  Whole_Array_Test[counter2][4]= np.sum(Recall)/5
  Whole_Array_Test[counter2][5]= np.sum(Precision)/5
  Whole_Array_Test[counter2][6]= np.sum(F1S)/5

  print(Whole_Array_Test[counter2])
    
  counter2 = counter2 + 1

print(Whole_Array_Test)
#Results = pd.DataFrame(Whole_Array_Test)
#Results.to_csv('Model4PCA.csv') 
#files.download('Model4PCA.csv')

In [ ]:
#Mixed, Admitted vs Not, Pre vs Post
#Patient 1 pre shock => 2 Hours Each
#Patient 2 post shock, in this case P1 and P2 are the same
P1times = datetime.strptime('1809202015450000000', '%d%m%Y%H%M%S%f')##########
P1timef = datetime.strptime('1809202017450000000', '%d%m%Y%H%M%S%f') 

P2times = datetime.strptime('18092020181500000', '%d%m%Y%H%M%S%f')##########
P2timef = datetime.strptime('18092020201500000', '%d%m%Y%H%M%S%f')

#Patient 3 had no shock
#Patient 4 had shock 
P3times = datetime.strptime('20072020143000000', '%d%m%Y%H%M%S%f')
P3timef = datetime.strptime('20072020163000000', '%d%m%Y%H%M%S%f')

P4times = datetime.strptime('03112020103000000', '%d%m%Y%H%M%S%f')
P4timef = datetime.strptime('03112020123000020', '%d%m%Y%H%M%S%f') #added 20 miliseconds to match the length of Patient 1


P5times = datetime.strptime('28072020160400000', '%d%m%Y%H%M%S%f')########## Start Until Shock
P5timef = datetime.strptime('29072020032820200', '%d%m%Y%H%M%S%f') 

P6times = datetime.strptime('29072020033146280', '%d%m%Y%H%M%S%f')########## Shock Until the End
P6timef = datetime.strptime('18092020044800000', '%d%m%Y%H%M%S%f') 


#Extracting


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

Patient1 = Raw[(Raw.study_no == '003-2104') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2104') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]
Patient3 = Raw[(Raw.study_no == '003-2162') & (P3times <= Raw.PPG_Datetime) & (P3timef > Raw.PPG_Datetime)]
Patient4 = Raw[(Raw.study_no == '003-2028') & (P4times <= Raw.PPG_Datetime) & (P4timef > Raw.PPG_Datetime)]
Patient5 = Raw[(Raw.study_no == '003-2009') & (P5times <= Raw.PPG_Datetime) & (P5timef > Raw.PPG_Datetime)]
Patient6 = Raw[(Raw.study_no == '003-2009') & (P6times <= Raw.PPG_Datetime) & (P6timef > Raw.PPG_Datetime)]


if Terminal:
  print(Patient1)
  print(Patient2)
  print(Patient3)
  print(Patient4)
  print(Patient5)
  print(Patient6)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 120)
Patient2_w = np.array_split(Patient2, 120)
Patient3_w = np.array_split(Patient3, 120)
Patient4_w = np.array_split(Patient4, 120)
Patient5_w = np.array_split(Patient5, 684)
Patient6_w = np.array_split(Patient6, 77)

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient_w),6000,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'PLETH_bpf') #Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'PLETH_bpf')
Patient3_w_arr = choose_signal(Patient3_w, 'PLETH_bpf') 
Patient4_w_arr = choose_signal(Patient4_w, 'PLETH_bpf')
Patient5_w_arr = choose_signal(Patient5_w, 'PLETH_bpf') 
Patient6_w_arr = choose_signal(Patient6_w, 'PLETH_bpf')


Patient1_w_arr_l = np.zeros((120,1), dtype=int)
Patient2_w_arr_l = np.ones((120,1), dtype=int)
Patient3_w_arr_l = np.full((120,1),2, dtype=int)
Patient4_w_arr_l = np.full((120,1),3, dtype=int)
Patient5_w_arr_l = np.full((684,1),4, dtype=int)
Patient6_w_arr_l = np.full((77,1),5, dtype=int)

#Defining Sampling rate
fs = 100

Score_array = np.empty((5,1))

##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr[-77:,:],Patient2_w_arr[:77,:],Patient3_w_arr[-77:,:],Patient4_w_arr[-77:,:],Patient5_w_arr[-77:,:],Patient6_w_arr))
Labels = np.vstack((Patient1_w_arr_l[-77:,:],Patient2_w_arr_l[:77,:],Patient3_w_arr_l[-77:,:],Patient4_w_arr_l[-77:,:],Patient5_w_arr_l[-77:,:],Patient6_w_arr_l))

#making the dataset balanced

Recall = np.empty((5,1))
Precision = np.empty((5,1))
F1S = np.empty((5,1))


Whole_Array_Test = np.empty((30,7))

counter2 = 0
for jj in range(80,81,80):
  counter = 0
  k = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
  for train_index, test_index in k.split(Data,Labels):
    print("Fold Number =================================================== ", counter)
    X_train = Data[train_index]
    X_test = Data[test_index]
    y_train = Labels[train_index]
    y_test = Labels[test_index]

    if Terminal:
      print(train_index)
      print(test_index)
    #Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

    #Training_sets = np.vstack((P1_tr,P2_tr))
    Globalmean = np.mean(X_train)
    Globalstd = np.std(X_train)


    P_train = (X_train - Globalmean) / Globalstd
    P_test = (X_test - Globalmean) / Globalstd

    #Empty arrays
    #P1Sxx = np.empty((len(P1_training),26,32))
    #P1Sxxtest = np.empty((len(P1_test),26,32))
    PSxx = np.empty((len(P_train),26,26))
    PSxxtest = np.empty((len(P_test),26,26))


    #Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
    #Patient 1

    def spectrogram_arr(Input, Output_arr, fs):
      for i in range(len(Input)):
        f, t, Sxx = signal.spectrogram(Input[i], fs)
        fmin = 0 # Hz
        fmax = 10 # Hz
        freq_slice = np.where((f >= fmin) & (f <= fmax))
        # keep only frequencies of interest
        f   = f[freq_slice]
        Sxx = Sxx[freq_slice,:][0]
        Output_arr[i] = Sxx
      return Output_arr

    X_train = spectrogram_arr(P_train, PSxx, fs)
    X_test = spectrogram_arr(P_test, PSxxtest, fs)

    Terminal = False

    if Terminal:
      print(X_train.shape)
      print(y_train.shape)
      print(X_test.shape)
      print(y_test.shape)



    #MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
    #tf.random.set_seed(1234)

    #converting to categorical
    y_trainf = tf.keras.utils.to_categorical(y_train)
    y_testf = tf.keras.utils.to_categorical(y_test)
    #X_trainf = X_train.reshape(-1, 26*32)
    #X_testf = X_test.reshape(-1, 26*32)

    from sklearn.decomposition import PCA


    # #print(X_train)
    # print(X_test.shape)
    # pca = PCA(n_components=19) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    # for i in range(X_train.shape[2]):
    #   pca = pca.fit(X_train[:,:,i])
    #   X_trainf = pca.transform(X_train[:,:,i])
    #   X_testf = pca.transform(X_test[:,:,i])

    pca = PCA(n_components=jj) #No. OF COMPONENTS CHOSEN EMPIRICALLY
    X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
    X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
    pca = pca.fit(X_train_2d)
    X_trainf = pca.transform(X_train_2d)
    X_testf = pca.transform(X_test_2d)

    Terminal = True
    print(X_trainf.shape)
    print(X_testf.shape)
    
    if Terminal:
      print('Train Labels')
      print(y_trainf.shape)
      print('Train Data')
      print(X_train.shape)

    # Simple ANN Start ========================================================================================

    model = Sequential()
    model.add(Dense(256,'relu', input_shape=(jj,), kernel_initializer='glorot_uniform'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(512,'relu', bias_regularizer = l2(0.001) ,kernel_initializer='glorot_uniform'))
    model.add(Dense(512,'relu', bias_regularizer = l2(0.001) , kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(256,'relu', bias_regularizer = l2(0.001) , kernel_initializer='glorot_uniform'))
    model.add(Dense(6))
    model.add(Activation('Softmax'))

    # . . . 
    #early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
    model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    history = model.fit(X_trainf, y_trainf, epochs=700, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])



    # ANN ==========================================================================================

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Accuracy and Loss')
    plt.ylabel('accuracy/loss')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
    plt.show()

    score = model.evaluate(X_testf, y_testf)

    y_prob= model.predict(X_testf)
    y_pred_unseen = y_prob.argmax(axis=-1)

    PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

    print('\nConfusion Matrix on Unseen Data:')
    print(confusion_matrix(y_test, y_pred_unseen))

    print('\nReport on Unseen Data:')
    print(classification_report(y_test, y_pred_unseen))


    #Sourced from https://www.codegrepper.com/code-examples/python/frameworks/-file-path-python/roc+python+example
    # probs = model.predict_proba(X_testf)
    # preds = probs[:,1]
    # fpr, tpr, threshold = roc_curve(y_test, preds)
    # roc_auc = auc(fpr, tpr)

    # plt.title('Receiver Operating Characteristic on Unseen Data')
    # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    # plt.legend(loc = 'lower right')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()

    Score_array[counter] = score[1]
    Recall[counter] = PRF[1]
    Precision[counter] = PRF[0] 
    F1S[counter] =  PRF[2]
    counter = counter + 1

  print(Score_array)

  print("\n K-Fold CV Max Test Accuracy:")
  print(np.max(Score_array))

  print("\n K-Fold CV Min Test Accuracy:")
  print(np.min(Score_array))

  print("\n K-Fold CV Average Test Accuracy:")
  print(np.sum(Score_array)/5)
  print("\n K-Fold CV Average Test Recall:")
  print(np.sum(Recall)/5)
  print("\n K-Fold CV Average Test Precision:")
  print(np.sum(Precision)/5)
  print("\n K-Fold CV Average Test F1Score:")
  print(np.sum(F1S)/5)
  Whole_Array_Test[counter2][0]= jj
  Whole_Array_Test[counter2][1]= np.max(Score_array)
  Whole_Array_Test[counter2][2]= np.min(Score_array)
  Whole_Array_Test[counter2][3]= np.sum(Score_array)/5
  Whole_Array_Test[counter2][4]= np.sum(Recall)/5
  Whole_Array_Test[counter2][5]= np.sum(Precision)/5
  Whole_Array_Test[counter2][6]= np.sum(F1S)/5

  print(Whole_Array_Test[counter2])
    
  counter2 = counter2 + 1

print(Whole_Array_Test)
# Results = pd.DataFrame(Whole_Array_Test)
# Results.to_csv('Model5PCA.csv') 
# files.download('Model5PCA.csv')